In [ ]:
from bs4 import BeautifulSoup
import ujson as json
import requests

In [ ]:
list_url = "http://terms.naver.com/list.nhn?cid=51007&page=%d&categoryId=51007"
mobile_root = "http://m.terms.naver.com"
entries = []

page = 1
while True:
    print(page)
    
    res = requests.get(list_url%page)
    b = BeautifulSoup(res.text, 'lxml')
    
    current_page = int(b.find('div', {'class': 'paginate'}).find('strong').text)
    if page > current_page:
        # stop if there is no more pages to crawl
        break
    
    for entry in b.find('ul', {'class': 'content_list'}).findAll('li'):
        url = entry.findAll('a')[0]['href']
        res = requests.get(mobile_root+url)
        b = BeautifulSoup(res.text, 'lxml')
        
        entry = dict()
        title = None

        div_content = b.find('div', {'class': '_SLOG_TERMS_CONTENT'})
        additional_info = div_content.find('ul', {'class': 'na_property_table2'})
        if additional_info is not None:
            for li in additional_info.findAll('li'):
                entry[li.find('em').text] = li.find('span').text

        div_content = div_content.findAll('div', {'class': 'atomic-block'})
        title_group = div_content[0].find('div', {'class': 'na_title_group'})
        entry['ko_name'] = title_group.find('h1').text
        try:
            entry['name'] = title_group.find('p').text.replace('\xa0', ' ')
        except:
            pass
        
        abstract = div_content[0].find('div', {'class': 'na_abstract'})
        if abstract is not None:
            abstract.find('strong').extract()
            entry['abstract'] = abstract.text.strip()
            
        for div in div_content[1:]:
            if "breakable-block" in div['class']:
                entry[title] = div.text.strip()
            else:
                title = div.findChild()
                if 'na_section_title' in title['class']:
                    title = div.text
                else:
                    # picture div
                    continue
                
        entries.append(entry)

    page += 1

In [ ]:
with open('result.json', 'w') as f:
    json.dump(entries, f)